In [1]:
import os
import glob
import cv2
import random
from PIL import Image
from tqdm import tqdm

def save_to_frame(input_dir, save_dir, category):
    """
    input:
        input_dir: directory to the video files
        save_dir: directory to save the frames converted from the video
    output:
        None
    """
    os.makedirs(save_dir, exist_ok=True)
    for path in tqdm(glob.glob(input_dir + '/*')):
        fname = os.path.basename(path).split('.')[0]
#         os.makedirs(os.path.join(save_dir, fname), exist_ok=True)
        vidcap = cv2.VideoCapture(path)
        success, image = vidcap.read()
        count = 0
        while success:
            if count % 1 == 0:
                #print("count = {}".format(count))
                cv2.imwrite("{}/{}-{}.jpg".format(save_dir, fname, str(count).zfill(4)), image)     # save frame as JPEG file            
            success, image = vidcap.read()
            count += 1


In [2]:
save_to_frame("./SCVD/Videos/Testing/Violence", "./SCVD/Frames/Testing/Violence", "violence")

100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [18:42<00:00, 15.81s/it]


In [3]:
save_to_frame("./SCVD/Videos/Training/Violence", "./SCVD/Frames/Training/Violence", "violence")

100%|██████████████████████████████████████████████████████████████████████████████| 165/165 [1:00:05<00:00, 21.85s/it]


In [4]:
save_to_frame("./SCVD/Videos/Testing/NonViolence", "./SCVD/Frames/Testing/NonViolence", "nonViolence")

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [06:28<00:00,  5.25s/it]


In [5]:
save_to_frame("./SCVD/Videos/Training/NonViolence", "./SCVD/Frames/Training/NonViolence", "nonViolence")

100%|████████████████████████████████████████████████████████████████████████████████| 172/172 [17:59<00:00,  6.28s/it]


In [1]:
#data preprocessing for feeding into lstm
import cv2
import os
dataset=[]
def load_images_from_folder(folder,x):
    image=[]
    im=[]
    c=1
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        n_img=cv2.resize(img,(100,100))
        if img is not None :
            im.append(n_img)
        if c%10==0:
            image.append(im)
            image.append(x)
            im=[]
            dataset.append(image)
            image=[]
        c=c+1
        
load_images_from_folder("./SCVD/Frames/Training/NonViolence",0)
load_images_from_folder("./SCVD/Frames/Training/Violence",1)

In [2]:
print(len(dataset))
# print(dataset)

import random
train_data = []
test_data = []

6207


In [3]:
import random
random.shuffle(dataset)
for samples in dataset[:10]:
    print(samples[1])
x=[]
y=[]
for img,lab in dataset:
    x.append(img)
    y.append(lab)

1
0
1
1
1
1
1
1
1
1


In [4]:
import numpy as np
x=np.array(x).reshape(-1,10,100,100,3)
print(x.shape)

(6207, 10, 100, 100, 3)


In [5]:
#saving everything as pickle
import pickle
pickle_out=open("xhockey.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()


pickle_out=open("yhockey.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

In [6]:
#loading from pickle
import pickle
pickle_in=open("xhockey.pickle","rb")
x=pickle.load(pickle_in)
pickle_in.close()

pickle_in=open("yhockey.pickle","rb")
y=pickle.load(pickle_in)
pickle_in.close()

In [7]:
#data preprocessing for feeding into lstm
import cv2
import os
validset=[]
def load_images_from_folder(folder,x):
    image=[]
    im=[]
    c=1
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        n_img=cv2.resize(img,(100,100))
        if img is not None :
            im.append(n_img)
        if c%10==0:
            image.append(im)
            image.append(x)
            im=[]
            validset.append(image)
            image=[]
        c=c+1 
load_images_from_folder("./SCVD/Frames/Testing/NonViolence",0)
load_images_from_folder("./SCVD/Frames/Testing/Violence",1)

In [8]:
print(len(validset))

2624


In [9]:
import random
random.shuffle(validset)
for samples in validset[:10]:
    print(samples[1])
x_valid=[]
y_valid=[]
for img,lab in validset:
    x_valid.append(img)
    y_valid.append(lab)

0
1
0
1
1
0
1
1
0
1


In [10]:
import numpy as np
x_valid=np.array(x_valid).reshape(-1,10,100,100,3)
print(x_valid.shape)

(2624, 10, 100, 100, 3)


In [11]:
x_valid=x_valid.astype('float32')/255
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

In [12]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import MobileNetV2

# K.image_data_format('tf')
import numpy as np
x=x.astype('float32')/255
NAME="SCVD"
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

mobilenet = MobileNetV2(
    input_shape=None,
#     alpha=1.0,
    include_top= False,
    weights=None,
    input_tensor=None,
    pooling=None,
#     classes=3,
#     classifier_activation="softmax"
)

## training the CNN
cnn = Sequential()
cnn.add(mobilenet)

#converting to 1-d tensor
cnn.add(Flatten())
model=Sequential()
model.add(TimeDistributed(cnn,input_shape=x.shape[1:]))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.25))
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(128,activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 10, 11520)        2257984   
 ibuted)                                                         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               2957568   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)               

In [ ]:
x = np.array(x)
y = np.array(y)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

model.fit(x,y,epochs=30,validation_data=(x_valid,y_valid),batch_size=20, callbacks=[tensorboard])
model.save("MobiBiLSTM-SCVD.h5")

In [ ]:
print(model.summary())